In [10]:
from pyspark.sql import SparkSession
import getpass
username=getpass.getuser()
spark=SparkSession.\
    builder.\
    config('spark.ui.port','0').\
    config("spark.sql.warehouse.dir",f"/user/{username}/warehouse").\
    enableHiveSupport().\
    master('yarn').\
    getOrCreate()

In [11]:
order_base = spark.sparkContext.textFile("source_file_path")

In [12]:
orders_mapped = order_base.map(lambda x: (x.split(",")[3],x.split(",")[2]))

In [13]:
customer_base = spark.sparkContext.textFile("/public/trendytech/retail_db/customers/part-00000")

In [14]:
customer_map = customer_base.map(lambda x: (x.split(",")[0],x.split(",")[8]))

In [15]:
customer_broadcast = spark.sparkContext.broadcast(cust_map.collect())

In [16]:
def get_pincode(customer_id):
    try:
        return customer_broadcast.value[customer_id]
    except:
        return "-1"

In [17]:
joined_rdd = orders_mapped.map(lambda x:(get_pincode(int(x[0])),x[1]))